# **Set according to environment (e.g. local, Google Colab...)**

In [1]:
project_folder = ''

# **Body**

In [ ]:
from custom_libraries.image_dataset import *
from custom_libraries.miscellaneous import *
import numpy as np

In [3]:
def create_model(num_units):
    model = tf.keras.Sequential()
    model.add(
        tf.keras.layers.Dense(units=num_units, activation=None, kernel_initializer=tf.keras.initializers.HeNormal))
    model.add(tf.keras.layers.LeakyReLU(alpha=.01))
    model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=['acc'])

    return model

In [4]:
bs = 256
trials = 10
epochs = 2000
trees_set = [1, 2, 4, 8, 16, 32]

classes = np.load(project_folder + 'results/classes.npy', allow_pickle=True)
# TODO: verificare l'effetto di patience sul risultato di svhn (rischio underfitting)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=60)

history = np.zeros((len(classes), trials, len(trees_set), 2))

#history = np.load(project_folder+'results/fcnn_history.npy', allow_pickle=True)
#history[-2,:,:,:] = 0

for j, (t1, t2, ds) in enumerate(classes):

    print(f"Dataset: {ds} / Pair: {t1}-{t2}")

    if history[j, 0, 0, 0] != 0:
        continue

    test_ds = ImageDataset(ds, 'test', USPS_dir=project_folder + 'USPS/')
    train_ds = ImageDataset(ds, 'train', USPS_dir=project_folder + 'USPS/')

    for x in [train_ds, test_ds]:
        x.filter(t1, t2, overwrite=True)
        x.shuffle()
        x.normalize()
        if x.images.shape[1:3] == (28, 28):
            x.pad()
        x.vectorize(True)

    X_train, y_train, X_valid, y_valid = train_ds.subset(shard=True, shard_number=trials, validation=True,
                                                         validation_size=len(test_ds.images))
    test_set = tf.data.Dataset.from_tensor_slices((test_ds.images, test_ds.labels)).batch(bs)

    for k, trees in enumerate(trees_set):

        print(f"{trees}-FCNN")

        for i in range(trials):

            if history[j, i, k, 0] != 0:
                continue

            print(f"Trial {i + 1}")

            model = create_model(num_units=2 * trees)

            train_set = tf.data.Dataset.from_tensor_slices((X_train[i], y_train[i])).batch(bs)
            valid_set = tf.data.Dataset.from_tensor_slices((X_valid[i], y_valid[i])).batch(bs)

            with tf.device('/device:GPU:0'):

                fit_history = model.fit(x=train_set, batch_size=bs, epochs=epochs,
                                        validation_data=valid_set, validation_batch_size=bs,
                                        callbacks=[callback], verbose=0)
                print_fit_history(fit_history, epochs)

                evaluate_history = model.evaluate(x=test_set, batch_size=bs, verbose=0)
                print_evaluate_history(evaluate_history)

                history[j, i, k] = evaluate_history

                np.save(project_folder + 'results/fcnn_history.npy', history,
                        allow_pickle=True)

Dataset: mnist / Pair: 3-5
Dataset: fmnist / Pair: 0-6
Dataset: kmnist / Pair: 2-6
Dataset: emnist / Pair: 14-17
Dataset: svhn / Pair: 5-6
Dataset: usps / Pair: 3-5
1-FCNN
Trial 1
Epochs: 2000/2000 - Train loss: 35.93%, accuracy: 95.51% - Validation loss: 45.42%, accuracy: 78.79%
Test loss: 44.72%, accuracy: 83.44%
Trial 2
Epochs: 2000/2000 - Train loss: 38.47%, accuracy: 97.75% - Validation loss: 40.03%, accuracy: 90.91%
Test loss: 43.3%, accuracy: 88.96%
Trial 3
Epochs: 2000/2000 - Train loss: 38.75%, accuracy: 100.0% - Validation loss: 43.4%, accuracy: 87.88%
Test loss: 45.52%, accuracy: 86.2%
Trial 4
Epochs: 2000/2000 - Train loss: 40.54%, accuracy: 94.38% - Validation loss: 46.57%, accuracy: 84.85%
Test loss: 43.23%, accuracy: 89.57%
Trial 5
Epochs: 2000/2000 - Train loss: 34.75%, accuracy: 95.51% - Validation loss: 43.32%, accuracy: 87.88%
Test loss: 48.42%, accuracy: 79.45%
Trial 6
Epochs: 2000/2000 - Train loss: 39.11%, accuracy: 89.89% - Validation loss: 45.08%, accuracy: 78.7

In [5]:
# Eseguire analisi statistica per confronto con Jones&Kording, Table 2, righe n-FCNN
print(history)

[[[[0.2609567  0.94532073]
   [0.15470001 0.94689798]
   [0.15390344 0.94794953]
   [0.15529346 0.95425868]
   [0.14052084 0.95636171]
   [0.12704054 0.96161938]]

  [[0.15896896 0.9584648 ]
   [0.11082867 0.95268136]
   [0.11478651 0.95268136]
   [0.10812325 0.95951629]
   [0.09437659 0.96792847]
   [0.08930919 0.96898001]]

  [[0.12778383 0.9584648 ]
   [0.1328606  0.95478445]
   [0.12545343 0.96004206]
   [0.10370782 0.96582544]
   [0.10322416 0.96687698]
   [0.0897609  0.96950579]]

  [[0.10260254 0.96056783]
   [0.10496873 0.9584648 ]
   [0.09170153 0.96319664]
   [0.09293742 0.96214509]
   [0.08273209 0.96950579]
   [0.07715058 0.97213459]]

  [[0.13745581 0.95162988]
   [0.14262661 0.9511041 ]
   [0.1281227  0.95320714]
   [0.14441228 0.95057833]
   [0.13990891 0.95373291]
   [0.12807661 0.96529967]]

  [[0.21488567 0.94164038]
   [0.16266315 0.95951629]
   [0.14738387 0.94637221]
   [0.12947215 0.95425868]
   [0.12344126 0.9584648 ]
   [0.11539809 0.96740276]]

  [[0.11075818 0